# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [2]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [3]:
# Checking the current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a "for loop" to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
print(file_path_list)
    

/home/workspace
['/home/workspace/event_data/2018-11-10-events.csv', '/home/workspace/event_data/2018-11-06-events.csv', '/home/workspace/event_data/2018-11-11-events.csv', '/home/workspace/event_data/2018-11-22-events.csv', '/home/workspace/event_data/2018-11-15-events.csv', '/home/workspace/event_data/2018-11-21-events.csv', '/home/workspace/event_data/2018-11-17-events.csv', '/home/workspace/event_data/2018-11-18-events.csv', '/home/workspace/event_data/2018-11-02-events.csv', '/home/workspace/event_data/2018-11-23-events.csv', '/home/workspace/event_data/2018-11-01-events.csv', '/home/workspace/event_data/2018-11-20-events.csv', '/home/workspace/event_data/2018-11-30-events.csv', '/home/workspace/event_data/2018-11-08-events.csv', '/home/workspace/event_data/2018-11-24-events.csv', '/home/workspace/event_data/2018-11-13-events.csv', '/home/workspace/event_data/2018-11-16-events.csv', '/home/workspace/event_data/2018-11-05-events.csv', '/home/workspace/event_data/2018-11-26-events.c

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [4]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:
    print(f)

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
# extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

/home/workspace/event_data/2018-11-10-events.csv
/home/workspace/event_data/2018-11-06-events.csv
/home/workspace/event_data/2018-11-11-events.csv
/home/workspace/event_data/2018-11-22-events.csv
/home/workspace/event_data/2018-11-15-events.csv
/home/workspace/event_data/2018-11-21-events.csv
/home/workspace/event_data/2018-11-17-events.csv
/home/workspace/event_data/2018-11-18-events.csv
/home/workspace/event_data/2018-11-02-events.csv
/home/workspace/event_data/2018-11-23-events.csv
/home/workspace/event_data/2018-11-01-events.csv
/home/workspace/event_data/2018-11-20-events.csv
/home/workspace/event_data/2018-11-30-events.csv
/home/workspace/event_data/2018-11-08-events.csv
/home/workspace/event_data/2018-11-24-events.csv
/home/workspace/event_data/2018-11-13-events.csv
/home/workspace/event_data/2018-11-16-events.csv
/home/workspace/event_data/2018-11-05-events.csv
/home/workspace/event_data/2018-11-26-events.csv
/home/workspace/event_data/2018-11-14-events.csv
/home/workspace/even

In [5]:
# check the number of rows in csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


#### Creating a Cluster

In [6]:
# Make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
try:
     cluster = Cluster(['127.0.0.1'])
     session = cluster.connect()  # To establish connection and begin executing queries, need a session
except Exception as e:
    print(e)


#### Create Keyspace

In [7]:
# Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity WITH REPLICATION = { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }""")

except Exception as e:
    print(e)


#### Set Keyspace

In [8]:
# Set KEYSPACE to the keyspace specified above

try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)


In [9]:
## Create a Table music_library with the PARTITION KEY and Clustering Columns as sessionId, itemInSession, firstName
## Note: Here the Primary Key has two fields: sessionId is the partition key, and itemInSession are clustering keys. Partitioning is done by sessionId and within that partition, \
## rows are ordered by the itemInSession.
## Run the query to get the artist details in the music app:
query = "CREATE TABLE IF NOT EXISTS music_library"
query = query + "(artist text, firstName text, gender text, itemInSession int, lastName text, length double, level text, location text, sessionId int, song text, userId int, \
PRIMARY KEY(sessionId, itemInSession))"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

In [10]:
# Write the code to set up the CSV file
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO music_library (artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId)"
        query = query + "VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
## Assign which column element should be assigned for each column in the INSERT statement
        session.execute(query, (line[0], line[1], line[2], int(line[3]), line[4], float(line[5]), line[6], line[7], int(line[8]), line[9], int(line[10])))

In [11]:
## Run the query to get the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
## Add in the SELECT statement to verify the data was entered into the table

query = "select artist,song,length from music_library WHERE sessionId = 338 and itemInSession = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
for row in rows:
    print (row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.3073


In [12]:
## Create Table song_playlist_session with Partition Key as userId, sessionId and Clustering Column as itemInSession with CLUSTERING ORDER BY itemInSession
## Run the query to get the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

query = "CREATE TABLE IF NOT EXISTS song_playlist_session"
query = query + "(artist text, firstName text, gender text, itemInSession int, lastName text, length decimal, level text, location text, sessionId int, song text, \
                userId int, PRIMARY KEY((userId, sessionId), itemInSession)) WITH CLUSTERING ORDER BY (itemInSession DESC)"
try:
    session.execute(query)
except Exception as e:
    print(e)
    


In [13]:
# Write the code to set up the CSV file

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        #print(line)
## Assigning the INSERT statements into the `query` variable
        query = "INSERT INTO song_playlist_session (artist,firstName,gender,itemInSession,lastName,length, level,location,sessionId,song,userId)"
        query = query + "VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
## Assigning which column element should be assigned for each column in the INSERT statement
        session.execute(query, (line[0], line[1], line[2], int(line[3]), line[4], float(line[5]), line[6], line[7], int(line[8]), line[9], int(line[10])))


In [14]:
## Run the query to get the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

query = "select artist,song,firstName,lastName from song_playlist_session WHERE userId = 10 AND sessionId = 182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
for row in rows:
    print("Artist Name - ", row.artist," Song - ", row.song, " FirstName - ", row.firstname," LastName - ", row.lastname)

Artist Name -  Lonnie Gordon  Song -  Catch You Baby (Steve Pitron & Max Sanna Radio Edit)  FirstName -  Sylvie  LastName -  Cruz
Artist Name -  Sebastien Tellier  Song -  Kilometer  FirstName -  Sylvie  LastName -  Cruz
Artist Name -  Three Drives  Song -  Greece 2000  FirstName -  Sylvie  LastName -  Cruz
Artist Name -  Down To The Bone  Song -  Keep On Keepin' On  FirstName -  Sylvie  LastName -  Cruz


In [ ]:
## Create Table music_app_history with Partition Key as song and Clustering Column as userId
query = "CREATE TABLE IF NOT EXISTS music_app_history "
query = query + "(artist text, firstName text, gender text, itemInSession int, lastName text, length decimal, level text, location text, \
sessionId int, song text, userId int, PRIMARY KEY(song, userId))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [ ]:
# Write the code to set up the CSV file
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        #print(line)
## Assigning the INSERT statements into the `query` variable
        query = "INSERT INTO music_app_history (artist,firstName,gender,itemInSession,lastName,length, level,location,sessionId,song,userId)"
        query = query + "VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
## Assigning which column element should be assigned for each column in the INSERT statement
        session.execute(query, (line[0], line[1], line[2], int(line[3]), line[4], float(line[5]), line[6], line[7], int(line[8]), line[9], int(line[10])))

In [ ]:
## Run the query to get every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
query = "select firstName,lastName from music_app_history WHERE song='All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
for row in rows:
    print( " FirstName - ", row.firstname," LastName - ", row.lastname)

### Drop the tables before closing out the sessions

In [ ]:
## Drop the table before closing out the sessions
query = "drop table music_library"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)


In [ ]:
query = "drop table song_playlist_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)


In [ ]:
query = "drop table music_app_history"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)


### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()